[ RAG 구현 절차 ]

```
1.	문서의 내용을 읽는다(document_loader를 이용)
(1)	https://python.langchain.com/v0.2/docs/integrations/document_loaders/ 
(2)	https://python.langchain.com/v0.2/docs/integrations/document_loaders/microsoft_word/
%pip install --upgrade --quiet  docx2txt
2.	문서를 쪼갠다(한번에 이해하고 처리할 수 있는 입력+출력 토큰수가 제한)
(1)	 https://python.langchain.com/v0.2/docs/how_to/recursive_text_splitter/#splitting-text-from-languages-without-word-boundaries 
%pip install -qU langchain-text-splitters
3.	쪼갠 문서를 임베딩하여 vector database에 넣음
(1)	OpenAIEmbeddings나 UpstageEmbeddings이용해서 임베딩
(2)	https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/  
%pip install –q langchain-chroma
4.	질문을 이용해 유사도 검색
5.	유사도 검색한 문서를 LLM에 질문으로 전달하여 답변 얻음(제공되는 Prompt활용)
(1)	https://python.langchain.com/v0.2/docs/tutorials/rag/
%pip install –q langchain langchainhub
http://smith.langchain.com에서 key생성 .env key(LANGCHAIN_API_KEY) 추가

```

# 0. 패키지 설치

In [1]:
# 문서 읽어오기
%pip install --quiet  docx2txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
# 텍스트를 청크로 나누는 기능만 있는 경량 모듈
%pip install -q langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.


In [3]:
# 벡터DB (로컬DB)
%pip install -q langchain-chroma

Note: you may need to restart the kernel to use updated packages.


In [4]:
# 제공되는 prompt 사용
%pip install -q langchain langchainhub

Note: you may need to restart the kernel to use updated packages.


# 1. 문서 읽기(X)

In [5]:
%%time
from langchain_community.document_loaders import Docx2txtLoader
loader = Docx2txtLoader('./tax_docs/소득세법(법률)(제20615호)(20250701).docx')
document = loader.load()

CPU times: total: 4.11 s
Wall time: 4.32 s


In [6]:
len(document)

1

In [7]:
document[0].page_content[:200]

'소득세법\n\n소득세법\n\n[시행 2025. 7. 1.] [법률 제20615호, 2024. 12. 31., 일부개정]\n\n기획재정부(재산세제과(양도소득세)) 044-215-4312\n\n기획재정부(소득세제과(근로소득)) 044-215-4216\n\n기획재정부(금융세제과(이자소득, 배당소득)) 044-215-4233\n\n기획재정부(소득세제과(사업소득, 기타소득)) 044-2'

# 2. 문서를 쪼개면서 읽기(o)

In [8]:
import time
start = time.time()
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader = Docx2txtLoader('./tax_docs/소득세법(법률)(제20615호)(20250701).docx')
text_splitter = RecursiveCharacterTextSplitter(  # 문서를 쪼개는 기준이 문자수
    chunk_size=1500, #문서를 쪼갤때 1500글자씩 쪼개
    chunk_overlap=200
)
# 1번째 chunk 1~1450글자
# 2번째 chunk 1250~1750글자
document = loader.load_and_split(text_splitter=text_splitter)
runtime = time.time() - start
print('문서 쪼개면서 읽는 시간 :', runtime)

문서 쪼개면서 읽는 시간 : 4.5747950077056885


In [9]:
# chunk 갯수
len(document)

183

In [10]:
len(document[0].page_content)

1464

In [11]:
# chunk의 글자수들
# [len(doc.page_content) for doc in document]
print(max(len(doc.page_content) for doc in document))
print(min(len(doc.page_content) for doc in document))

1497
1055


# 3. 쪼갠문서를 임베딩 -> 벡터 데이터베이스 저장
- 임베딩 모델 : openAI API의 text-embedding-3-large (기본:text-embedding-ada-002)
- 벡터 데이터베이스 : chroma

In [12]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
load_dotenv()
# https://python.langchain.com/v0.2/docs/how_to/embed_text/
embedding = OpenAIEmbeddings(
    model = "text-embedding-3-large"
)

In [13]:
embeddings = embedding.embed_documents(
    [
        "소득세법 어쩌구 저쩌구",
        document[0].page_content
    ]
)
len(embeddings), len(embeddings[0])

(2, 3072)

In [14]:
len(embeddings), len(embeddings[0]), len(embeddings[1])

(2, 3072, 3072)

In [15]:
len(embedding.embed_query("소득세"))

3072

In [16]:
%%time
from langchain_chroma import Chroma
#데이터를 처음 저장할 때
database = Chroma.from_documents(                                 
    documents=document,
    embedding=embedding,
    collection_name="tax-collection", # 생략시 이름 랜덤
    persist_directory='./chroma'      # 생략시 로컬데이터베이스에 저장안됨. 프로그램 종료시 db날라감
)


CPU times: total: 1.61 s
Wall time: 7.32 s


In [17]:
# 이미 저장된 vector DB를 사용할 때
database = Chroma(
    embedding_function=embedding,
    collection_name="tax-collection",
    persist_directory='./chroma'
)

# 4. vector DB에 질문과 유사도 검색(답변 생성을 위한 retrieval)

In [18]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
retrieved_docs = database.similarity_search(query,
                                           k=3) # 기본 k는 4

In [19]:
retrieved_docs

[Document(id='70c3e481-13fb-44c9-9f1a-2f264a3abd62', metadata={'source': './tax_docs/소득세법(법률)(제20615호)(20250701).docx'}, page_content='[전문개정 2009. 12. 31.]\n\n\n\n제10조(납세지의 변경신고) 거주자나 비거주자는 제6조부터 제9조까지의 규정에 따른 납세지가 변경된 경우 변경된 날부터 15일 이내에 대통령령으로 정하는 바에 따라 그 변경 후의 납세지 관할 세무서장에게 신고하여야 한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제11조(과세 관할) 소득세는 제6조부터 제10조까지의 규정에 따른 납세지를 관할하는 세무서장 또는 지방국세청장이 과세한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2장 거주자의 종합소득 및 퇴직소득에 대한 납세의무 <개정 2009. 12. 31.>\n\n\n\n제1절 비과세 <개정 2009. 12. 31.>\n\n\n\n제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 20., 2018. 3. 20., 2018. 12. 31., 2019. 12. 10., 2019. 12. 31., 2020. 6. 9., 2020. 12. 29., 2022. 8. 12., 2022. 12. 31., 2023. 8. 8., 2023. 12. 31., 2024. 12. 31.>\n\n1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n\n나. 1개의 주택을 소유하는 자의 주택임대소득(

# 5. 유사도 검색으로 가져온 문서를 질문과 같이 LLM 전달하여 답변 생성

In [20]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4.1-nano")

In [21]:
prompt = f"""[identity]
- 당신은 최고의 한국 소득세 전문자입니다
- [context]를 참고해서 사용자의 질문에 답변해 주세요
[context]는 다음과 같아요
{retrieved_docs}
Question : {query}"""

In [22]:
ai_message = llm.invoke(prompt)

In [23]:
print(ai_message.content)

연봉이 5천만 원인 직장인의 소득세는 다음과 같이 계산됩니다. 

1. 총급여액(연봉): 50,000,000원
2. 근로소득 공제: 근로소득자에 대한 기본 공제 및 근로소득 공제표에 따라 공제액이 산출됩니다. 
   - 일반적으로 연봉 5천만 원인 경우, 근로소득 공제는 약 1,700만 원 내외로 추정됩니다. 구체적인 금액은 근로소득공제 표를 참고하여 계산해야 하며, 대략 1,760만 원 정도 예상됩니다.
3. 과세표준: 총급여액 - 근로소득공제 - 인적공제(본인 공제, 부양가족 공제 등) 
   - 본인 인적공제는 150만 원입니다.
   - 부양가족(예, 배우자, 자녀 등)이 있다면 해당 공제도 적용됩니다.
   
예를 들어, 부양가족이 없다면:

- 과세표준 = 50,000,000 - 17,600,000 - 1,500,000(본인 공제) ≈ 30,900,000원

4. 세율 적용: 과세표준에 따른 세율표를 적용합니다.

대략적인 세율표 (2023년 세법 기준):

| 과세표준 (원) | 세율(%) | 누진세액(원) |
|----------------|---------|--------------|
| 1,200만원 이하 | 6%      |  과세표준×6% |
| 1,200만원 초과 ~ 4,600만원 이하 | 15% | 1,200만원×6% + (과세표준-1200만원)×15% |
| 4,600만원 초과 ~ 8,800만원 이하 | 24% | 1,200만원×6% + (4,600만원-1200만원)×15% + (과세표준-4600만원)×24% |
| ... | | |

이번 과세표준 30,900,000원에 적용하면:

- 1,200만원까지: 12,000,000×6% = 720,000원
- 나머지: 30,900,000 - 12,000,000 = 18,900,000원

  *4,600만원 초과 구간 해당 없음*, 따라서:

- 4,600만원까지까지의 세금 계산으로:

  1,200만원×6% = 720,000원
  (4,600만원 - 1,200만원)×15% = 3,400만

# 5. Augmentation을 위한 제공되는 Prompt 활용하여 langchain으로 답변 생성

In [24]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

from langchain import hub

prompt = hub.pull('rlm/rag-prompt')
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

### RetrivalQA를 통해 LLM전달 (create_retrival_chain)이 대체
```
    query -> retriever전달 (벡터 검색 수행) -> retriever문서 -> prompt에 {context}에 삽입 -> query -> prompt의 {question}에 삽입 
```

In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = database.as_retriever(search_kwargs={"k": 3}),
    chain_type_kwargs={"prompt": prompt},
)

ValidationError: 2 validation errors for RetrievalQA
retriever
  Field required [type=missing, input_value={'combine_documents_chain...search_kwargs={'k': 3})}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
retrieval
  Extra inputs are not permitted [type=extra_forbidden, input_value=VectorStoreRetriever(tags... search_kwargs={'k': 3}), input_type=VectorStoreRetriever]
    For further information visit https://errors.pydantic.dev/2.11/v/extra_forbidden

In [26]:
message = qa_chain.invoke({"query":query})
print(message)

NameError: name 'qa_chain' is not defined